<a href="https://colab.research.google.com/github/JPleal01/Dio_IA/blob/main/metricas_de_verificacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Carregar bibliotecas e dataset para treinamento
#dataset bem conhecido, números escritos para classificação, principal intuíto é calcular as métricas
from tensorflow.keras import datasets, layers, models
from tensorflow import math, argmax
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd

%load_ext tensorboard
logdir = 'log'
(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()

In [ ]:
#redimensionando as imagens
train_images = train_images.reshape((60000, 28, 28, 1))
test_images = test_images.reshape((10000, 28, 28, 1))

train_images, test_images = train_images / 255.0, test_images / 255.0
classes = [0,1,2,3,4,5,6,7,8,9]

In [ ]:
#Treinamento do modelo
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPool2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPool2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=5, validation_data=(test_images, test_labels))

In [ ]:
y_true = test_labels
y_pred = model.predict(test_images)

In [ ]:
#Montando matriz de confusão
predicted_labels = argmax(y_pred, axis=1)
con_mat = math.confusion_matrix(labels=y_true,predictions=predicted_labels).numpy()
con_mat_norm = np.around(con_mat.astype('float') / con_mat.sum(axis=1)[:, np.newaxis], decimals=4)

con_mat_df = pd.DataFrame(con_mat_norm,
                          index = classes,
                          columns = classes)

figure = plt.figure(figsize=(8, 8))
sns.heatmap(con_mat_df, annot=True,cmap=plt.cm.Blues)
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

In [ ]:
# Desenvolver logica para calculo das variaveis solicitadas.

#from sklearn.metrics import classification_report
#predicted_labels = argmax(y_pred, axis=1)
#print(classification_report(y_true, predicted_labels, target_names=[str(i) for i in classes]))

for classe in con_mat_df.index:
    TP = con_mat_df.loc[classe, classe]
    FP = con_mat_df[classe].sum() - TP
    FN = con_mat_df.loc[classe].sum() - TP
    TN = con_mat_df.values.sum() - (TP + FP + FN)

sensibilidade = TP / (TP + FN)
especificidade = TN / (TN + FP)
acuracia = (TP + TN) / (TP + TN + FP + FN)
precisao = TP / (TP + FP)
f1_score = 2 * (precisao * sensibilidade) / (precisao + sensibilidade)

print(f"Sensibilidade: {sensibilidade}")
print(f"Especificidade: {especificidade}")
print(f"Acurácia: {acuracia}")
print(f"Precisão: {precisao}")
print(f"F-Score: {f1_score}")




In [ ]:
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from tensorflow.keras.utils import to_categorical

# Número de classes
n_classes = len(classes) # Use the length of the classes list

# One-hot encode the true labels
y_true_one_hot = to_categorical(y_true, num_classes=n_classes)

# Gráfico
plt.figure(figsize=(10, 8))

for i in range(n_classes):
    fpr, tpr, _ = roc_curve(y_true_one_hot[:, i], y_pred[:, i])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, lw=2, label=f'Classe {i} (AUC = {roc_auc:.2f})')

# Linha de referência (modelo aleatório)
plt.plot([0, 1], [0, 1], 'k--', lw=2)

# Estética do gráfico
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Taxa de Falsos Positivos')
plt.ylabel('Taxa de Verdadeiros Positivos')
plt.title('Curvas ROC - Multiclasse (One-vs-Rest)')
plt.legend(loc="lower right")
plt.grid(True)
plt.tight_layout()
plt.show()